In [1]:
from multiprocessing import Pool
import numpy as np
import pyvinecopulib as pv

from vcimpute.helper_datagen import make_complete_data_matrix, mask_MCAR
from gcimpute.helper_evaluation import get_smae
from itertools import permutations

In [2]:
d = 10
X = make_complete_data_matrix(1000, d, 'gaussian', seed=10)
X_mask = mask_MCAR(X, 'monotone', .2, seed=10, n_cols=2)

In [21]:
np.count_nonzero(np.isnan(X_mask), axis=0)

array([  0,   0,   0,   0,   0,   0, 358,   0,   0,  42])

In [4]:
controls = pv.FitControlsVinecop(family_set=[pv.BicopFamily.gaussian])

In [22]:
def process(p):
    X_imp = np.copy(X_mask)
    structure = pv.DVineStructure(order=list(p)) # can just be any valid vine structure
    cop = pv.Vinecop(structure)
    cop.select(X_imp, controls)
    rb = cop.rosenblatt(X_imp)
    rb = np.ndarray.flatten(rb)
    nan_idx = np.flatnonzero(np.isnan(X_imp))
    rb[nan_idx] = np.random.uniform(size=len(nan_idx)) # replace rng
    rb = rb.reshape(X_imp.shape)
    nan_rows = np.any(np.isnan(X_imp), axis=1)
    X_imp[nan_rows, :] = cop.inverse_rosenblatt(rb[nan_rows, :])
    return (p, get_smae(X_imp, X, X_mask)[6], get_smae(X_imp, X, X_mask)[9])